In [1]:
import pyodbc
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import random
import numpy as np
import tensorflow as tf

server = 'cyse492group8.database.windows.net'
database = 'CYSE492_Project'
username = 'cyseadmin'
password = 'GroupProject492'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [51]:
cursor.execute("SELECT [Control Identifier],[Control Text]  FROM [dbo].['NIST_800.53_controls$']")

labels = []
texts = []
for row in cursor:
    texts.append(row[1])
    if 'AC' in row[0]:
        labels.append(1)
    if 'AT' in row[0]:
        labels.append(2)
    if 'AU' in row[0]:
        labels.append(3)
    if 'CA' in row[0]:
        labels.append(4)
    if 'CM' in row[0]:
        labels.append(5)
    if 'CP' in row[0]:
        labels.append(6)
    if 'IA' in row[0]:
        labels.append(7)
    if 'IR' in row[0]:
        labels.append(8)
    if 'MA' in row[0]:
        labels.append(9)
    if 'MP' in row[0]:
        labels.append(10)
    if 'PE' in row[0]:
        labels.append(11)
    if 'PL' in row[0]:
        labels.append(12)
    if 'PM' in row[0]:
        labels.append(13)
    if 'PS' in row[0]:
        labels.append(14)
    if 'PT' in row[0]:
        labels.append(15)
    if 'RA' in row[0]:
        labels.append(16)
    if 'SA' in row[0]:
        labels.append(17)
    if 'SC' in row[0]:
        labels.append(18)
    if 'SI' in row[0]:
        labels.append(19)
    if 'SR' in row[0]:
        labels.append(20)

x_test = []
y_test = []
x_act = np.array(texts)
y_act = np.array(labels)

# test is the actual test, act is used for the training
for i in range(150):
    random_index = random.randrange(len(labels))
    x_test.append(texts.pop(random_index))
    y_test.append(labels.pop(random_index))
x_test = np.array(x_test)
y_test = np.array(y_test)

In [3]:
# Hyperparameters
max_words = 15000 # max number of words to use in the vocabulary was 15k
max_len = 400 # max length of each text (in terms of number of words) 250
embedding_dim = 400 # dimension of word embeddings was 100
lstm_units = 32 # number of units in the LSTM layer
# num_classes = len(set(labels)) + 1 # number of classes
num_classes = len(set(y_act)) + 1 # number of classes
#num_classes2 = len(set(y_test)) + 1

# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_act)

# Pad the sequences so they all have the same length
x = pad_sequences(sequences, maxlen=max_len)


# Create one-hot encoded labels
y = keras.utils.to_categorical(y_act, num_classes)

# y_test = keras.utils.to_categorical(y_test, num_classes)
# Build the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(lstm_units))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x, y, batch_size=32, epochs=20)# was 32,20




Epoch 1/20


38/38 [==============================] - 19s 322ms/step - loss: 2.9127 - accuracy: 0.1346
Epoch 2/20
38/38 [==============================] - 10s 260ms/step - loss: 2.5873 - accuracy: 0.2389
Epoch 3/20
38/38 [==============================] - 11s 287ms/step - loss: 2.1514 - accuracy: 0.3734
Epoch 4/20
38/38 [==============================] - 10s 268ms/step - loss: 1.7165 - accuracy: 0.5357
Epoch 5/20
38/38 [==============================] - 10s 267ms/step - loss: 1.3751 - accuracy: 0.6602
Epoch 6/20
38/38 [==============================] - 11s 287ms/step - loss: 1.0796 - accuracy: 0.7468
Epoch 7/20
38/38 [==============================] - 10s 276ms/step - loss: 0.8380 - accuracy: 0.8158
Epoch 8/20
38/38 [==============================] - 11s 275ms/step - loss: 0.6782 - accuracy: 0.8537
Epoch 9/20
38/38 [==============================] - 11s 294ms/step - loss: 0.5829 - accuracy: 0.8831
Epoch 10/20
38/38 [==============================] - 10s 270ms/step - loss: 0.4896 - ac

In [ ]:
###padding
## reuse old tokenizer for x_test
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_test)

## Pad the sequences so they all have the same length
x_test = pad_sequences(sequences, maxlen=max_len)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Test the model     
#If you want to test different x_test/y_test you can compile that section instead of retraining the whole thing again
accr = model.evaluate(x_test,y_test) #x and y instead of test?
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#for i in range(10):
#    result = tf.argmax(model.predict(tf.expand_dims(texts[i], 0)), axis=1)
#    print(result.numpy(), labels[i])

In [170]:
#Train with CIS
#Get names of tables
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES")

cisNames = []
for names in cursor:
    if names[0]!=("NIST_800.53_controls$") or names[0]!="NIST_800.53_baselines$":
        cisNames.append(names[0])
print(cisNames[0])

#labels is the section num, texts is the description
labels = []
texts = []
#Get individual tables
for count in range(len(cisNames)):
    try:
        cursor.execute("SELECT [Description]  FROM [dbo].["+cisNames[count]+"]")
        #checks and sorts by first number in the section number
        for row in cursor:
            
            texts.append(str(row[0]))       
            # if '1' in row[0][0]:
            #     labels.append(row[0])
            # if '2' in row[0][0]:
            #     labels.append(row[0])            
            # if '3' in row[0][0]:
            #     labels.append(row[0])
            # if '4' in row[0][0]:
            #     labels.append(row[0])
            # if '5' in row[0][0]:
            #     labels.append(row[0])
            # if '6' in row[0][0]:
            #     labels.append(row[0])
            # if '7' in row[0][0]:
            #     labels.append(row[0])
            # if '8' in row[0][0]:
            #     labels.append(row[0])
            # if '9' in row[0][0]:
            #     labels.append(row[0])                    
            labels.append(count+1)
    except:
        print(cisNames[count]+" is missing either description or label. Skipping")

x_test = []
y_test = []
x_act = np.array(texts)
y_act = np.array(labels)

# test is the actual test, act is used for the training
for i in range(150):
    random_index = random.randrange(len(labels))
    x_test.append(texts.pop(random_index))
    y_test.append(labels.pop(random_index))
x_test = np.array(x_test)
y_test = np.array(y_test)


'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 Combined Profiles$'
'NIST_800.53_controls$' is missing either description or label. Skipping
'MITRE ATT&CK Filtering$' is missing either description or label. Skipping
'NIST_800.53_baselines$' is missing either description or label. Skipping
'MITRE ATT&CK Filtering$'_xlnm#_FilterDatabase is missing either description or label. Skipping
CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 License$ is missing either description or label. Skipping
'MITRE ATT&CK Mappings$' is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 MITRE ATT&CK Filtering$' is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 MITRE ATT&CK Filtering$'_xlnm#_FilterDatabase is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 MITRE ATT&CK Mappings$' is missing either description or label. Skipping
'CIS_Apple_macOS_12.0_Monterey_Benchmark_v3.0.0 O

In [171]:
print(len((x_act)))
print(len((y_act)))
print(len(cisNames))

71206
71206
280


In [172]:
##reset params
num_classes = len(set(y_act)) +1 # number of classes
#num_classes2 = len(set(y_test)) + 1

# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_act)

# Pad the sequences so they all have the same length
x = pad_sequences(sequences, maxlen=max_len)


# Create one-hot encoded labels
#######THIS IS THE LINE CAUSING THE ERROR IDK HOW FIX
y = keras.utils.to_categorical(y_act,num_classes)

# Train the model again with CIS
history = model.fit(x, y, batch_size=32, epochs=20)# was 32,20


IndexError: index 187 is out of bounds for axis 1 with size 187

In [ ]:
#new tests
###padding
## reuse old tokenizer for x_test
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_act)
sequences = tokenizer.texts_to_sequences(x_test)

## Pad the sequences so they all have the same length
x_test = pad_sequences(sequences, maxlen=max_len)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Test the model     
#If you want to test different x_test/y_test you can compile that section instead of retraining the whole thing again
accr = model.evaluate(x_test,y_test) #x and y instead of test?
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#for i in range(10):
#    result = tf.argmax(model.predict(tf.expand_dims(texts[i], 0)), axis=1)
#    print(result.numpy(), labels[i])

In [ ]:
print(x_test.shape)
print(y_test.shape)
print(x.shape)
print(y.shape)
print(x_test)
print(y_test)
print(x)
print(y)
print(x_act[1])

In [ ]:
print(len(set(y_act)))